In [ ]:
# evaluation test with ALBEF for VQA

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import
import argparse
import os
import ruamel.yaml as yaml
import numpy as np
import random
import time
import datetime
import json
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
import torch.distributed as dist

from models.model_vqa import ALBEF
from models.vit import interpolate_pos_embed
from models.tokenization_bert import BertTokenizer

import utils
from dataset.utils import save_result
from dataset import create_dataset, create_sampler, create_loader, vqa_collate_fn

from scheduler import create_scheduler
from optim import create_optimizer

In [3]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import zipfile

# prep downloaded data
VQA_DATA_DIR = 'data/vqa'

zip_files = [
    'train2014.zip',
    'test2015.zip',
    'v2_Annotations_Train_mscoco.zip',
    'v2_Annotations_Val_mscoco.zip',
    'v2_Complementary_Pairs_Train_mscoco.zip',
    'v2_Complementary_Pairs_Val_mscoco.zip',
    'v2_Questions_Test_mscoco.zip',
    'v2_Questions_Train_mscoco.zip',
    'v2_Questions_Val_mscoco.zip',
    'val2014.zip'
]

for zip_file in zip_files:
    zip_path = os.path.join(VQA_DATA_DIR, zip_file)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(VQA_DATA_DIR)


In [ ]:
# set up VQA config
args = argparse.Namespace()
args.config = './configs/VQA.yaml'
args.checkpoint = './ALBEF_4M.pth'
args.output_dir = './output/vqa'
args.evaluate = False
args.text_encoder = 'bert-base-uncased'
args.text_decoder = 'bert-base-uncased'
args.device = 'cuda'
args.seed = 42
args.distributed = False

config = yaml.load(open(args.config, 'r'), Loader=yaml.Loader)
print(args)

In [7]:
args.result_dir = os.path.join(args.output_dir, 'result')

Path(args.output_dir).mkdir(parents=True, exist_ok=True)
Path(args.result_dir).mkdir(parents=True, exist_ok=True)

yaml.dump(config, open(os.path.join(args.output_dir, 'config.yaml'), 'w'))

In [8]:
utils.init_distributed_mode(args)

device = torch.device(args.device)

# fix the seed for reproducibility
seed = args.seed + utils.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
cudnn.benchmark = True

start_epoch = 0
max_epoch = config['schedular']['epochs']
warmup_steps = config['schedular']['warmup_epochs']

Not using distributed mode


In [9]:
#### Dataset ####
print("Creating vqa datasets")
datasets = create_dataset('vqa', config)

if args.distributed:
    num_tasks = utils.get_world_size()
    global_rank = utils.get_rank()
    samplers = create_sampler(datasets, [True, False], num_tasks, global_rank)
else:
    samplers = [None, None]

train_loader, test_loader = create_loader(datasets,samplers,
                                          batch_size=[config['batch_size_train'],config['batch_size_test']],
                                          num_workers=[4,4],is_trains=[True, False],
                                          collate_fns=[vqa_collate_fn,None])

tokenizer = BertTokenizer.from_pretrained(args.text_encoder)

Creating vqa datasets


FileNotFoundError: [Errno 2] No such file or directory: 'data/vqa_train.json'